In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Celiac_Disease"
cohort = "GSE193442"

# Input paths
in_trait_dir = "../../input/GEO/Celiac_Disease"
in_cohort_dir = "../../input/GEO/Celiac_Disease/GSE193442"

# Output paths
out_data_file = "../../output/preprocess/Celiac_Disease/GSE193442.csv"
out_gene_data_file = "../../output/preprocess/Celiac_Disease/gene_data/GSE193442.csv"
out_clinical_data_file = "../../output/preprocess/Celiac_Disease/clinical_data/GSE193442.csv"
json_path = "../../output/preprocess/Celiac_Disease/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Transcriptional profiling of human KIR+ CD8 T cells"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['tissue: PBMC'], 1: ['cell type: KIR+ CD8 T']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information, this seems to be a dataset about transcriptional profiling
# which suggests gene expression data is likely available
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# From the sample characteristics dictionary, we don't see any clear indicators of 
# celiac disease status, age, or gender information

# For trait (Celiac Disease)
trait_row = None  # Not available in the sample characteristics

# For age
age_row = None  # Not available in the sample characteristics

# For gender
gender_row = None  # Not available in the sample characteristics

# Define conversion functions even though they won't be used in this case
def convert_trait(value):
    if value is None:
        return None
    value = value.split(": ")[-1].strip().lower()
    if "celiac" in value or "coeliac" in value:
        return 1
    elif "control" in value or "healthy" in value or "normal" in value:
        return 0
    return None

def convert_age(value):
    if value is None:
        return None
    try:
        # Extract the numeric part after colon
        age_str = value.split(": ")[-1].strip()
        # Try to convert to float
        return float(age_str)
    except:
        return None

def convert_gender(value):
    if value is None:
        return None
    value = value.split(": ")[-1].strip().lower()
    if "female" in value or "f" == value:
        return 0
    elif "male" in value or "m" == value:
        return 1
    return None

# 3. Save Metadata
# Determine if trait data is available
is_trait_available = trait_row is not None

# Conduct initial filtering and save cohort information
validate_and_save_cohort_info(
    is_final=False, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is None, we should skip this substep


False

### Step 3: Gene Data Extraction

### Step 4: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Check the file to understand its structure
print("This dataset is a SuperSeries, which is a collection of related datasets rather than containing gene expression data directly.")
print("A SuperSeries typically doesn't contain the actual gene data, but rather references to SubSeries that contain the data.")

# 3. Look for subseries information in the SOFT file
print("\nChecking for SubSeries information in the SOFT file...")
try:
    with gzip.open(soft_file, 'rt') as f:
        subseries_lines = []
        for line in f:
            if "!Series_relation" in line and "SuperSeries of:" in line:
                subseries_lines.append(line.strip())
            # Also look for subseries IDs directly
            elif "SubSeries" in line and "GSE" in line:
                subseries_lines.append(line.strip())
        
        if subseries_lines:
            print("Found SubSeries information:")
            for line in subseries_lines[:10]:  # Show up to 10 subseries
                print(f"  {line}")
            if len(subseries_lines) > 10:
                print(f"  ...and {len(subseries_lines) - 10} more")
        else:
            print("No explicit SubSeries information found in SOFT file.")
except Exception as e:
    print(f"Error reading SOFT file: {e}")

# 4. Since this is a SuperSeries without direct gene data, we need to set is_gene_available to False
print("\nThis SuperSeries doesn't contain gene expression data directly.")
print("To process gene expression data, you would need to:")
print("1. Identify the relevant SubSeries")
print("2. Download and process those individual datasets separately")
print("3. Combine the results as needed for your analysis")

# Set empty gene_data similar to what get_genetic_data returns for compatibility
import pandas as pd
gene_data = pd.DataFrame(index=pd.Index([], name='ID'))
print(f"\nGene data shape: {gene_data.shape}")
print("First 20 gene/probe identifiers:")
print(gene_data.index[:20])

# 5. Update the is_gene_available flag for step 2
print("\nBased on this analysis, is_gene_available should be set to False for this dataset.")

Matrix file found: ../../input/GEO/Celiac_Disease/GSE193442/GSE193442-GPL18573_series_matrix.txt.gz
This dataset is a SuperSeries, which is a collection of related datasets rather than containing gene expression data directly.
A SuperSeries typically doesn't contain the actual gene data, but rather references to SubSeries that contain the data.

Checking for SubSeries information in the SOFT file...
Found SubSeries information:
  !Series_relation = SuperSeries of: GSE168527
  !Series_relation = SuperSeries of: GSE193439
  !Series_relation = SuperSeries of: GSE193770

This SuperSeries doesn't contain gene expression data directly.
To process gene expression data, you would need to:
1. Identify the relevant SubSeries
2. Download and process those individual datasets separately
3. Combine the results as needed for your analysis

Gene data shape: (0, 0)
First 20 gene/probe identifiers:
Index([], dtype='object', name='ID')

Based on this analysis, is_gene_available should be set to False fo